## Assymetric Cryptography

In [1]:
import random
import math
import codecs
import base64

In [2]:
class Utils:
    def __init__(self):
        '''if not used then delete'''
    
    def hex_to_dec(hex):
        deci = int(hex, 16)
        return deci

    def dec_to_hex(dec):
        hexa = hex(dec)
        return hexa

    def hex_to_b64(hex):
        hex = hex[2:] if(len(hex[2:]) % 2 == 0) else '0' + hex[2:]
        b64 = codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()
        return b64
       
    def b64_to_hex(b64):
        hexa = base64.b64decode(b64).hex()
        return hexa
    
    def dec_to_b64(dec):
        b64 = Utils.hex_to_b64(Utils.dec_to_hex(dec))
        return b64

    def b64_to_dec(b64):
        deci = Utils.hex_to_dec(Utils.b64_to_hex(b64))
        return deci

    def is_square(i):
        return i == math.isqrt(i) ** 2

    def mod_sqrt(n, p):

        points = []
        n = n % p
        for x in range (1, p):
            if ((x ** 2) % p == n) :
                points.append(x)
        return points

        

In [3]:
class ElGamal:
    def __init__(self):
        pass

    def create_key(self, p, g, x):
        y = pow(g, x, p)
        public_key = { 'y': y, 'g': g, 'p': p }
        private_key = { 'x': x, 'p': p }
        return public_key, private_key

    def encrypt(self, message, key, public_key):
        a = pow(public_key['g'], key, public_key['p'])
        b = (pow(public_key['y'], key, public_key['p']) * (message % public_key['p'])) % public_key['p']
        return { 'a': a, 'b': b }

    def decrypt(self, message, private_key):
        return ((message['b'] % private_key['p']) * pow(message['a'], (private_key['p'] - 1 - private_key['x']), private_key['p'])) % private_key['p']

In [4]:
elgamal = ElGamal()

public_key, private_key = elgamal.create_key(2357, 2, 1751)

print("Public Key\t: " + str(public_key))
print("Private Key\t: " + str(private_key))

encrypted = elgamal.encrypt(2035, 1520, public_key)
print("Ciphertext\t: " + str(encrypted))

decrypted = elgamal.decrypt(encrypted, private_key)
print("Plaintext\t: " + str(decrypted))

Public Key	: {'y': 1185, 'g': 2, 'p': 2357}
Private Key	: {'x': 1751, 'p': 2357}
Ciphertext	: {'a': 1430, 'b': 697}
Plaintext	: 2035


In [5]:
class ElGamalMachine:
    def __init__(self):
        self.elgamal = ElGamal()
    
    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_key(self, bit):
        p = self.create_random_prime(bit)
        g = random.randint(1, p - 1)
        x = random.randint(1, p - 2)

        public_key, private_key = self.elgamal.create_key(p, g, x)

        public_key['y'] = Utils.dec_to_b64(public_key['y'])
        public_key['g'] = Utils.dec_to_b64(public_key['g'])
        public_key['p'] = Utils.dec_to_b64(public_key['p'])

        private_key['x'] = Utils.dec_to_b64(private_key['x'])
        private_key['p'] = Utils.dec_to_b64(private_key['p'])

        return public_key, private_key

    def encrypt(self, message, public_key):
        y = Utils.b64_to_dec(public_key['y'])
        g = Utils.b64_to_dec(public_key['g'])
        p = Utils.b64_to_dec(public_key['p'])

        p_key = { 'y': y, 'g': g, 'p': p }

        encrypted = ''
        for m in message:
            k = random.randint(1, p-2)
            res = self.elgamal.encrypt(ord(m), k, p_key)
            a = Utils.dec_to_b64(res['a'])
            b = Utils.dec_to_b64(res['b'])
            encrypted += a + b
        return encrypted

    def decrypt(self, encrypted, private_key):
        x = Utils.b64_to_dec(private_key['x'])
        p = Utils.b64_to_dec(private_key['p'])

        p_key = { 'x': x, 'p': p }
        
        encrypted = encrypted.split('=\n')[:-1]
        encrypted_paired = []
        for i in range(0, len(encrypted), 2):
            a = Utils.b64_to_dec(encrypted[i] + '=\n')
            b = Utils.b64_to_dec(encrypted[i+1] + '=\n')
            encrypted_paired.append({ 'a': a, 'b': b })

        message = ''
        for m in encrypted_paired:
            message += chr(self.elgamal.decrypt(m, p_key))
        return message

In [6]:
machine = ElGamalMachine()

public_key, private_key = machine.create_key(256)

encrypted = machine.encrypt('Aku adalah = x +', public_key)
print(encrypted)
print(machine.decrypt(encrypted, private_key))

gtlxYM5YWS/zeoIDFIJcbNgAtQt8hjCJvECnpZItl8s=
FIK+amtl+QaEuf0tmxdzNk17GABuvUmr1nMhe9bXy5I=
NGajs6TpwRbwEiNRK6NFqY1inhgsWe1ne405hUwpOhc=
RmhydqIH2liayyxyLZb+gtMwyKRulysKURhu0koRdWw=
OYFsSjFBjU9iAYd4K23tpAKyTGn4Sma7u+jIyFHH5dY=
Oaxmia4QzrNY6ibJFz2xY7Gcw3jrc3hTHm0s/HS3ZTs=
G87xxSLN0EZDjt8ad+edbeiVzqO3eBmiB4evsohZD9Q=
RwcXu13ioYhEizLcVNVKNIM8kJ2fs3HAevwH+GtXI1o=
JQZgAuGw1CssPA7jw+8P64ZaDKmQNR4ExIiitguYGh0=
c52M6PqP/YTwFIkAO9V2UTV9M+S5D0AP7nm818j4mJw=
VoNhmmBF7xR4+qI/0PPANvEnmAgExxsVc5Gt4rRCsPg=
ryPzcKEVhMbpvWW+bZCgcUq4yoh/lsgQU0xJTn/4n0s=
Cm1tiucuXo73aeN9R7bBpxnEgnv0POBAr8hilGyFhRw=
omirsF7noJqjCjvKYuHxzb8ITbYKp9UswkCDPsawajY=
cHV3StYBGAOS2UGFsKR9yc9lZJ4AHIDLti0xHMdbcqI=
C4qEBGmXh5xdgiPqDNXFr9mF73/3VWy8ULiWdmDMpuk=
rr3g5uPC0tUf/IP14TNCjSz1ZpEdSh8S/UDiYpTD1/E=
W8YgZAfkeOERB9tW68VZ344gkrSQAYWYpa8UdGhRAx0=
N2OygEh3/7aHs2u4oAgmP4Ui8nl7ArTS3Wa34nIxql0=
DQyGuNsbD2mKCUMEXCP5dS69KGL4xPFN+LbApnS2DN0=
Kt7BMqRo2OHxpD7+UQfoo+AYL2l6nkYpm4cYU6UyNf0=
e+jQPOW4qsK7tXgcOKwAaN6WHddVOFoA0esIQsMaKN4=
m+t7KZLLnD

In [7]:
class EllipticPoint:
    def __init__(self, x=0, y=0, inf=False):
        self.x = x if not inf else math.inf
        self.y = y if not inf else math.inf
        self.inf = inf

    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def is_infinity(self):
        return self.inf
    
    def mirror(self):
        return EllipticPoint(self.x, -self.y, self.inf)

    def __eq__(self, point):
        if(isinstance(point, EllipticPoint)):
            if(self.inf):
                return self.inf == point.inf
            else:
                return self.x == point.x and self.y == point.y
            
        return False

    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)

In [8]:
class EllipticCurve:
    def __init__(self, a, b, p):
        ''' y^2 = x^3 + ax + b '''
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        if(self.is_singular()):
            print("Error: Curve is Singular!.")
    
    def is_singular(self):
        return((((4 * (self.a ** 3)) + (27 * (self.b ** 2))) % self.p) == 0)

    def get_points(self):
        points = [EllipticPoint(inf=True)]
        
        for x in range(self.p):
            y_square = x ** 3 + self.a * x + self.b

            for point in Utils.mod_sqrt(y_square % self.p, self.p):
                points.append(EllipticPoint(x, point))
        self.points = points 
    
    def get_y_square(self, x):
        y_square = x ** 3 + self.a * x + self.b

        for point in Utils.mod_sqrt(y_square % self.p, self.p):
            return(EllipticPoint(x, point))

    def point_addition(self, P, Q):
        if(P == Q):
            return self.point_multiplication(P)
        else:
            xp = P.get_x(); yp = P.get_y();
            xq = Q.get_x(); yq = Q.get_y();

            if(xp == xq):
                return EllipticPoint(inf=True)
            else:
                m = ((yp-yq) * pow((xp-xq), -1, self.p)) % self.p

                xr = (m ** 2 - xp - xq) % self.p
                yr = (m * (xp - xr) - yp) % self.p
                return EllipticPoint(xr, yr)

    def point_substraction(self, P, Q):
        return self.point_addition(P, Q.mirror())

    def point_multiplication(self, P):
        
        xp = P.get_x(); yp = P.get_y();

        if(yp == 0):
            return EllipticPoint(inf=True)
        else:
            m = ((3 * (xp ** 2) + self.a) * pow((2 * yp), -1, self.p)) % self.p
            xr = (m ** 2 - 2 * xp) % self.p
            yr = (m * (xp - xr) - yp) % self.p
            return EllipticPoint(xr, yr)

    def point_scalar_multiplication(self, P, k):
        if(k == 1):
            return P
        elif(k == 2):
            return self.point_multiplication(P)
        else:
            prev = P
            res = self.point_multiplication(P)
            for i in range(k-2):
                if(res.is_infinity()):
                    res = self.point_addition(prev, self.point_multiplication(P))
                else:
                    prev = res
                    res = self.point_addition(res, P)
    
            return res
    

In [9]:
curve = EllipticCurve(1, 6, 11)
curve.get_points()

# print("N: " + str(len(curve.points)))
# for x in curve.points:
#     print(x)

# print()

P = EllipticPoint(2, 4)
# Q = EllipticPoint(5, 9)
# print(curve.point_addition(P, Q))
# print(curve.point_multiplication(P))

# print()

for i in range(1, len(curve.points)):
    print(curve.point_scalar_multiplication(P, i))






(2, 4)
(5, 9)
(8, 8)
(10, 9)
(3, 5)
(7, 2)
(7, 9)
(3, 6)
(10, 2)
(8, 3)
(5, 2)
(2, 7)


In [10]:
class ECCElGamalMachine:
    def __init__(self):
        pass

    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_agreement(self, bit):
        p = self.create_random_prime(bit)
        curve_a = random.randint(-p, p)
        curve_b = random.randint(-p, p)

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        found = False

        B = None

        while(not found):
            B = curve.points[random.randint(1, len(curve.points) - 2)]
            if(not B.is_infinity()):
                found = True
        
        return p, curve_a, curve_b, B

    def create_key(self, p, curve_a, curve_b):




        a = random.randint(1, p-2)
        b = random.randint(1, p-2)


       
        public_key = {}
        private_key = {}

        return public_key, private_key




In [32]:
machine = ECCElGamalMachine()

p, curve_a, curve_b, B = machine.create_agreement(4)

print(p)
print(curve_a)
print(curve_b)
print(B)

IndexError: list index out of range

In [12]:
M = 11

li = [220, 224, 229]

k = 20

found = False
i = 1
while(not found):
    x = M * k + i
    if x in li:
        found = True
    i = i + 1

print(x)

decoded = math.floor((x-1)/k)
print(decoded)

224
11


In [13]:
# ALICE
# Kunci Privat = a
# Kunci Publik = PA = a . B

# BOB
# Kunci Privat = b
# Kunci Publik = PB = b . B

# Bilangan Acak k dalam interval [1, p-1]

# Ciphertext(PM) = PC = [(kB), (PM + kPB)]
# b . (kB) 


In [14]:
curve = EllipticCurve(-1, 16, 29)
curve.get_points()

P = EllipticPoint(5, 7)
Q = EllipticPoint(1, 25)
R = EllipticPoint(6, 20)


print(curve.point_multiplication(P))
print(curve.point_addition(Q, R))


(28, 4)
(23, 26)
